In [1]:

import os
import torch
from ultralytics import YOLO
from roboflow import Roboflow

# Check available GPUs
num_gpus = torch.cuda.device_count()
device_ids = list(range(num_gpus))  # Example: [0,1,2,3] if 4 GPUs are available

# Define dataset path (update if needed)
dataset_path = "/home/mansi/Sumit Kumar, Satyendra/Brain/Brain Aug 1"
data_yaml = os.path.join(dataset_path, "data.yaml")

# Check if dataset exists
if not os.path.exists(data_yaml):
    print("Dataset not found! Downloading...")
    rf = Roboflow(api_key="Rj7J0lDDcL5F3UabGv9S")
    project = rf.workspace("brain-o889o").project("brain-qicsj")
    version = project.version(2)
    dataset = version.download("yolov8")
    os.rename(dataset.location, dataset_path)
else:
    print("Dataset already exists. Using the existing dataset.")

# Load YOLO model
model = YOLO("yolov8n")  # Adjust model if needed




Dataset not found! Downloading...
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Brain-2 in yolov8:: 100%|█████████████████████████████████████████████████████████| 23736/23736 [00:03<00:00, 6946.82it/s]


In [2]:
import os
from collections import defaultdict

# Base dataset path
dataset_path = "/home/mansi/Sumit Kumar, Satyendra/Brain/Brain Aug 1"

# Corrected label paths
labels_dirs = [
    os.path.join(dataset_path, "valid", "labels"),
    os.path.join(dataset_path, "test", "labels"),
]

# Dictionary to track images per class
class_to_images = defaultdict(set)

# Iterate and count
for labels_dir in labels_dirs:
    if not os.path.exists(labels_dir):
        print(f"Missing directory: {labels_dir}")
        continue

    print(f"Scanning: {labels_dir}")
    for label_file in os.listdir(labels_dir):
        if not label_file.endswith(".txt"):
            continue

        label_path = os.path.join(labels_dir, label_file)

        with open(label_path, "r") as f:
            lines = f.readlines()
            for line in lines:
                if line.strip() == "":
                    continue
                class_id = int(line.split()[0])
                class_to_images[class_id].add(label_file)

# Output counts
print("\nNumber of images per class:")
for class_id in range(4):  # Classes 0 to 4
    count = len(class_to_images[class_id])
    print(f"Class {class_id}: {count} images")


Scanning: /home/mansi/Sumit Kumar, Satyendra/Brain/Brain Aug 1/valid/labels
Scanning: /home/mansi/Sumit Kumar, Satyendra/Brain/Brain Aug 1/test/labels

Number of images per class:
Class 0: 313 images
Class 1: 322 images
Class 2: 351 images
Class 3: 346 images


In [3]:
# Train with optimized settings
model.train(
    data=data_yaml,
    name="Brain aug v8",
    device=device_ids,  # Use all available GPUs
    epochs=100, 
    batch=64,  # Adjust batch size for better GPU utilization
    workers=16,  # Use multiple data loading workers
)

Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/mansi/Sumit Kumar, Satyendra/Brain/Brain Aug 1/data.yaml, epochs=100, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1, 2, 3], workers=16, project=None, name=Brain aug v8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_

train: Scanning /home/mansi/Sumit Kumar, Satyendra/Brain/Brain Aug 1/train/labels... 10530 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10530/10530 [00:14<00:00, 727.30it/s]


train: New cache created: /home/mansi/Sumit Kumar, Satyendra/Brain/Brain Aug 1/train/labels.cache


val: Scanning /home/mansi/Sumit Kumar, Satyendra/Brain/Brain Aug 1/valid/labels... 1000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:01<00:00, 902.09it/s]


val: New cache created: /home/mansi/Sumit Kumar, Satyendra/Brain/Brain Aug 1/valid/labels.cache
Plotting labels to runs/detect/Brain aug v8/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 64 dataloader workers
Logging results to runs/detect/Brain aug v8
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       1000       1000      0.979      0.977      0.991      0.914

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      1.97G     0.5753     0.3483     0.9302          9        640: 100%|██████████| 165/165 [00:25<00:00,  6.44it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:03<00:00,  8.78it/s]


                   all       1000       1000      0.979      0.977      0.991      0.914

100 epochs completed in 1.135 hours.
Optimizer stripped from runs/detect/Brain aug v8/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/Brain aug v8/weights/best.pt, 6.2MB

Validating runs/detect/Brain aug v8/weights/best.pt...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:05<00:00,  6.14it/s]


                   all       1000       1000      0.979      0.977      0.991      0.914
                     0        228        228       0.99      0.996      0.995      0.938
                     1        256        256       0.96      0.965      0.988      0.892
                     2        252        252       0.98       0.96      0.985      0.898
                     3        264        264      0.986      0.989      0.995      0.928
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/Brain aug v8


In [4]:
# Validation
print("Running validation...")
metrics = model.val()
print("Validation Results:", metrics)

Running validation...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/mansi/Sumit Kumar, Satyendra/Brain/Brain Aug 1/valid/labels.cache... 1000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/16 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 316.00 MiB. GPU 0 has a total capacity of 10.75 GiB of which 246.50 MiB is free. Process 31412 has 5.15 GiB memory in use. Process 17006 has 3.20 GiB memory in use. Including non-PyTorch memory, this process has 2.15 GiB memory in use. Of the allocated memory 896.01 MiB is allocated by PyTorch, and 231.99 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Testing
print("Running testing...")
test_results = model.val(split="test")
print("Test Results:", test_results)